In [ ]:
DATA_PATH = '../data/processed/01_preprocessed_df.pkl'
EXPORT_PATH = '../data/processed/02_cleaned_df.pkl'

ROLE_COLS  = ['DevType']
TECH_COLS  = ['LanguageHaveWorkedWith',
              'DatabaseHaveWorkedWith',
              'WebframeHaveWorkedWith',
              'MiscTechHaveWorkedWith',
              'ToolsTechHaveWorkedWith']

EXCLUDE_ROLES = ['Other (please specify):',
                 'Student',
                 'Designer',
                 'Educator',
                 'Marketing or sales professional',
                 'Engineering manager',
                 'Senior Executive (C-Suite, VP, etc.)',
                 'Product manager',
                 'Engineer, site reliability']

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from scripts.preprocessing import one_hot_encode

### Initialize and preprocess

In [ ]:
# Read and preprocess
processed_df = pd.read_pickle(DATA_PATH)
ohe_df = one_hot_encode(processed_df, ROLE_COLS + TECH_COLS)

In [ ]:
# Initialize exclusion list
exclude_masks = []

## Build filtering masks

### 1. Filter on number of roles specified

In [ ]:
# Calculate
freq_roles = ohe_df[ROLE_COLS].sum(axis=1)
freq_roles.value_counts().sort_index()

In [ ]:
# Create and save exclusion mask
N_ROLES_RANGE = [1,3]

exclude_nroles = (freq_roles < N_ROLES_RANGE[0]) | (freq_roles > N_ROLES_RANGE[1])
exclude_nroles.name = 'n_roles'

exclude_masks.append(exclude_nroles)

### 2. Filter on number of techs specified

In [ ]:
# Calculate
freq_tech = ohe_df[TECH_COLS].sum(axis=1)
freq_tech.value_counts().sort_index()

In [ ]:
# plot
plt.hist(freq_tech, freq_tech.max())
plt.title('Number of Skills')
plt.show()

In [ ]:
# Create and save exclusion mask
N_TECH_RANGE = [2,15]

exclude_ntech = (freq_tech < N_TECH_RANGE[0]) | (freq_tech > N_TECH_RANGE[1])
exclude_ntech.name = 'n_tech'

exclude_masks.append(exclude_ntech)

### 3. Filter on employment

In [ ]:
# Calculate
processed_df['Employment'].value_counts()

In [ ]:
# Create and save exclusion mask
KEEP_EMPLOYMENT = ['Employed full-time', 'Employed part-time']

exclude_employment = ~processed_df['Employment'].isin(KEEP_EMPLOYMENT)
exclude_employment.name = 'employment'

exclude_masks.append(exclude_employment)

### 4. Filter on relevant roles

In [ ]:
exclude_single_roles = [[role] for role in EXCLUDE_ROLES]

In [ ]:
exclude_role = processed_df['DevType'].isin(exclude_single_roles)
exclude_role.name = 'role'

exclude_masks.append(exclude_role)

_____

## Filter Dataframe

In [ ]:
# Merge masks
exclude_df = pd.DataFrame(exclude_masks).T

In [ ]:
# Get insights: percentages of exclusion per filter
exclude_df.sum() * 100 / len(exclude_df)

In [ ]:
# Get insights: multiple exclusion criteria
# 36% of the data will be kept
exclude_df.sum(axis=1).value_counts() * 100 / len(exclude_df)

In [ ]:
# If any of the exclusion criteria occured, drop
final_exclusion_mask = exclude_df.any(axis=1)
cleaned_df = processed_df[~final_exclusion_mask].copy()

## Export

In [ ]:
cleaned_df.to_pickle(EXPORT_PATH)